In [2]:
!pip install scikit-learn
!pip install pandas
!pip install pandas-datareader
!pip install matplotlib
!pip install pillow
!pip install requests
!pip install h5py
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install opencv-python
!pip install opencv-contrib-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for keras<2.16,>=2.15.0 from https://files.pythonhosted.org/packages/fc/a7/0d4490de967a67f68a538cc9cdb259bff971c4b5787f7765dc7c8f118f71/keras-2.15.0-py3-none-any.whl.metadata
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing in

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.utils import to_categorical  # Import to_categorical from keras.utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
import itertools
from sklearn.model_selection import train_test_split

In [15]:
#load dataset
x_train = np.load('trainX.npy') 
y_train = np.load('trainY.npy') 
x_test = np.load('testX.npy') 
y_test = np.load('testY.npy') 

# load the "Train Images"
#x_train = data['trainX']
#normalize every image
x_train = np.array(x_train,dtype='float32')/255

#x_test = data['testX']
x_test = np.array(x_test,dtype='float32')/255

# load the Label of Images
#y_train= data['trainY']
#y_test= data['testY']

# show the train and test Data format
print('x_train : {}'.format(x_train[:]))
print('Y-train shape: {}'.format(y_train))
print('x_test shape: {}'.format(x_test.shape))

x_train : [[0.1882353  0.19215687 0.1764706  ... 0.18431373 0.18039216 0.18039216]
 [0.23529412 0.23529412 0.24313726 ... 0.1254902  0.13333334 0.13333334]
 [0.15294118 0.17254902 0.20784314 ... 0.11372549 0.10196079 0.11372549]
 ...
 [0.44705883 0.45882353 0.44705883 ... 0.38431373 0.3764706  0.38431373]
 [0.4117647  0.4117647  0.41960785 ... 0.21176471 0.18431373 0.16078432]
 [0.45490196 0.44705883 0.45882353 ... 0.37254903 0.39215687 0.39607844]]
Y-train shape: [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1
  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7
  8  8  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 13 13
 14 14 14 14 14 14 1

In [16]:
x_train, x_valid, y_train, y_valid= train_test_split(
    x_train, y_train, test_size=.05, random_state=1234,)

In [17]:
im_rows=112
im_cols=92
batch_size=512
im_shape=(im_rows, im_cols, 1)

#change the size of images
x_train = x_train.reshape(x_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
x_valid = x_valid.reshape(x_valid.shape[0], *im_shape)

print('x_train shape: {}'.format(y_train.shape[0]))
print('x_test shape: {}'.format(y_test.shape))

x_train shape: 228
x_test shape: (160,)


In [18]:
#filters= the depth of output image or kernels

cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape= im_shape),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
     Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(20, activation='softmax')  
])

cnn_model.compile(
    loss='sparse_categorical_crossentropy',#'categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

In [19]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 106, 86, 36)       1800      
                                                                 
 max_pooling2d (MaxPooling2  (None, 53, 43, 36)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 49, 39, 54)        48654     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 24, 19, 54)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 24624)             0         
                                                                 
 dense (Dense)               (None, 2024)              4

In [20]:
history=cnn_model.fit(
    np.array(x_train), np.array(y_train), batch_size=512,
    epochs=40, verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)),
)

Epoch 1/40



1/1 - 5s - loss: 3.0018 - accuracy: 0.0351 - val_loss: 3.2823 - val_accuracy: 0.0000e+00 - 5s/epoch - 5s/step
Epoch 2/40
1/1 - 2s - loss: 3.2164 - accuracy: 0.0658 - val_loss: 2.9684 - val_accuracy: 0.0833 - 2s/epoch - 2s/step
Epoch 3/40
1/1 - 3s - loss: 3.0739 - accuracy: 0.0658 - val_loss: 2.9659 - val_accuracy: 0.0000e+00 - 3s/epoch - 3s/step
Epoch 4/40
1/1 - 3s - loss: 2.9821 - accuracy: 0.0570 - val_loss: 2.9858 - val_accuracy: 0.0000e+00 - 3s/epoch - 3s/step
Epoch 5/40
1/1 - 2s - loss: 2.9755 - accuracy: 0.1053 - val_loss: 2.9802 - val_accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 6/40
1/1 - 2s - loss: 2.9835 - accuracy: 0.0833 - val_loss: 2.9772 - val_accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 7/40
1/1 - 2s - loss: 2.9747 - accuracy: 0.0746 - val_loss: 2.9739 - val_accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 8/40
1/1 - 3s - loss: 2.9605 - accuracy: 0.1009 - val_loss: 2.9678 - val_accuracy: 0.0000e+00 - 3s/epoch - 3s/step
Epoch 9/40
1/1 - 3s - loss: 2.9602 - accuracy: 

In [23]:
scor = cnn_model.evaluate( np.array(x_test),  np.array(y_test), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

test los 0.4914
test acc 0.9062
